In [ ]:
#!pip install torch==1.4.0 torchvision==0.5.0
!pip install gym
!pip install gym[atari]

In [1]:
%matplotlib inline

import gym
import math
import random
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
from collections import namedtuple
from itertools import count

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torchvision.transforms as T

from continuous_cartpole import ContinuousCartPoleEnv 

from IPython import display
plt.ion()

# if gpu is to be used
device = torch.device("cuda")

In [3]:
from continuous_cartpole import ContinuousCartPoleEnv 
env = NormalizedEnv(ContinuousCartPoleEnv(False))

***

In [2]:
class NormalizedEnv(gym.ActionWrapper):
    """ Wrap action """

    def action(self, action):
        act_k = (self.action_space.high - self.action_space.low)/ 2.
        act_b = (self.action_space.high + self.action_space.low)/ 2.
        return act_k * action + act_b

    def reverse_action(self, action):
        act_k_inv = 2./(self.action_space.high - self.action_space.low)
        act_b = (self.action_space.high + self.action_space.low)/ 2.
        return act_k_inv * (action - act_b)

In [4]:
def plot_durations(episode_durations):
    fig, axs = plt.subplots(2, figsize=(10,10))
    
    durations_t, durations = list(map(list, zip(*episode_durations)))
    durations = torch.tensor(durations, dtype=torch.float)
    
    fig.suptitle('Training')
    axs[0].set_xlabel('Episode')
    axs[0].set_ylabel('Reward')
    
    axs[0].plot(durations_t, durations.numpy())
        
    plt.pause(0.001)  # pause a bit so that plots are updated
    display.clear_output(wait=True)

In [5]:
# [reference] https://github.com/matthiasplappert/keras-rl/blob/master/rl/random.py

class RandomProcess(object):
    def reset_states(self):
        pass

class AnnealedGaussianProcess(RandomProcess):
    def __init__(self, mu, sigma, sigma_min, n_steps_annealing):
        self.mu = mu
        self.sigma = sigma
        self.n_steps = 0

        if sigma_min is not None:
            self.m = -float(sigma - sigma_min) / float(n_steps_annealing)
            self.c = sigma
            self.sigma_min = sigma_min
        else:
            self.m = 0.
            self.c = sigma
            self.sigma_min = sigma

    @property
    def current_sigma(self):
        sigma = max(self.sigma_min, self.m * float(self.n_steps) + self.c)
        return sigma


# Based on http://math.stackexchange.com/questions/1287634/implementing-ornstein-uhlenbeck-in-matlab
class OrnsteinUhlenbeckProcess(AnnealedGaussianProcess):
    def __init__(self, theta, mu=0., sigma=1., dt=1e-2, x0=None, size=1, sigma_min=None, n_steps_annealing=1000):
        super(OrnsteinUhlenbeckProcess, self).__init__(mu=mu, sigma=sigma, sigma_min=sigma_min, n_steps_annealing=n_steps_annealing)
        self.theta = theta
        self.mu = mu
        self.dt = dt
        self.x0 = x0
        self.size = size
        self.reset_states()

    def sample(self):
        x = self.x_prev + self.theta * (self.mu - self.x_prev) * self.dt + self.current_sigma * np.sqrt(self.dt) * np.random.normal(size=self.size)
        self.x_prev = x
        self.n_steps += 1
        return x

    def reset_states(self):
        self.x_prev = self.x0 if self.x0 is not None else np.zeros(self.size)

In [6]:
def soft_update(target, source, tau):
    for target_param, param in zip(target.parameters(), source.parameters()):
        target_param.data.copy_(
            target_param.data * (1.0 - tau) + param.data * tau
        )

def hard_update(target, source):
    for target_param, param in zip(target.parameters(), source.parameters()):
            target_param.data.copy_(param.data)

In [7]:
# (state, action) -> (next_state, reward, done)
transition = namedtuple('transition', ('state', 'action', 'next_state', 'reward', 'done'))

# replay memory D with capacity N
class ReplayMemory(object):
    def __init__(self, capacity):
        self.capacity = capacity
        self.memory = []
        self.position = 0

    # implemented as a cyclical queue
    def store(self, *args):
        if len(self.memory) < self.capacity:
            self.memory.append(None)
        
        self.memory[self.position] = transition(*args)
        self.position = (self.position + 1) % self.capacity

    def sample(self, batch_size):
        return random.sample(self.memory, batch_size)

    def __len__(self):
        return len(self.memory)

***

In [8]:
def fanin_init(size, fanin=None):
    fanin = fanin or size[0]
    v = 1. / np.sqrt(fanin)
    return torch.Tensor(size).uniform_(-v, v)

class Actor(nn.Module):
    def __init__(self, nb_states, nb_actions):
        super(Actor, self).__init__()
        self.fc1 = nn.Linear(nb_states, 128)
        self.fc2 = nn.Linear(128, 128)
        self.head = nn.Linear(128, nb_actions)
        
        self.tanh = nn.Tanh()
        self.init_weights(3e-3)
    
    def init_weights(self, init_w):
        self.fc1.weight.data = fanin_init(self.fc1.weight.data.size())
        self.fc2.weight.data = fanin_init(self.fc2.weight.data.size())
        self.head.weight.data.uniform_(-init_w, init_w)
    
    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        return self.tanh(self.head(x))

class Critic(nn.Module):
    def __init__(self, nb_states, nb_actions):
        super(Critic, self).__init__()

        # Q1 architecture
        self.l1 = nn.Linear(nb_states + nb_actions, 128)
        self.l2 = nn.Linear(128, 128)
        self.l3 = nn.Linear(128, 1)

        # Q2 architecture
        self.l4 = nn.Linear(nb_states + nb_actions, 128)
        self.l5 = nn.Linear(128, 128)
        self.l6 = nn.Linear(128, 1)
    
    def forward(self, state, action):
        sa = torch.cat([state, action], 1).float()

        q1 = F.relu(self.l1(sa))
        q1 = F.relu(self.l2(q1))
        q1 = self.l3(q1)

        q2 = F.relu(self.l4(sa))
        q2 = F.relu(self.l5(q2))
        q2 = self.l6(q2)
        return q1, q2

    def Q1(self, state, action):
        sa = torch.cat([state, action], 1).float()

        q1 = F.relu(self.l1(sa))
        q1 = F.relu(self.l2(q1))
        q1 = self.l3(q1)
        return q1

In [9]:
BATCH_SIZE = 64
GAMMA = 0.99

# https://spinningup.openai.com/en/latest/algorithms/td3.html
class TD3(nn.Module):
    def __init__(self, nb_states, nb_actions):
        super(TD3, self).__init__()
        self.nb_states = nb_states
        self.nb_actions= nb_actions
        
        self.actor = Actor(self.nb_states, self.nb_actions)
        self.actor_target = Actor(self.nb_states, self.nb_actions)
        self.actor_optimizer  = optim.Adam(self.actor.parameters(), lr=0.0001)

        self.critic = Critic(self.nb_states, self.nb_actions)
        self.critic_target = Critic(self.nb_states, self.nb_actions)
        self.critic_optimizer  = optim.Adam(self.critic.parameters(), lr=0.0001)

        hard_update(self.actor_target, self.actor)
        hard_update(self.critic_target, self.critic)
        
        #Create replay buffer
        self.memory = ReplayMemory(100000)
        self.random_process = OrnsteinUhlenbeckProcess(size=nb_actions, theta=0.15, mu=0.0, sigma=0.2)

        # Hyper-parameters
        self.tau = 0.005
        self.depsilon = 1.0 / 50000
        self.policy_noise=0.2
        self.noise_clip=0.5
        self.policy_freq=2
        self.total_it = 0

        # 
        self.epsilon = 1.0
        self.s_t = None # Most recent state
        self.a_t = None # Most recent action
        self.is_training = True

    def update_policy(self):
        if len(self.memory) < BATCH_SIZE:
            return

        self.total_it += 1
        
        # in the form (state, action) -> (next_state, reward, done)
        transitions = self.memory.sample(BATCH_SIZE)
        batch = transition(*zip(*transitions))
        
        state_batch = torch.cat(batch.state)
        next_state_batch = torch.cat(batch.next_state)
        action_batch = torch.cat(batch.action)
        reward_batch = torch.cat(batch.reward)
        done_mask = np.array(batch.done)
        not_done_mask = torch.from_numpy(1 - done_mask).float().to(device)

        # Target Policy Smoothing
        with torch.no_grad():
            # Select action according to policy and add clipped noise
            noise = (
                torch.randn_like(action_batch) * self.policy_noise
            ).clamp(-self.noise_clip, self.noise_clip).float()
            
            next_action = (
                self.actor_target(next_state_batch) + noise
            ).clamp(-1.0, 1.0).float()

            # Compute the target Q value
            # Clipped Double-Q Learning
            target_Q1, target_Q2 = self.critic_target(next_state_batch, next_action)
            target_Q = torch.min(target_Q1, target_Q2).squeeze(1)
            target_Q = (reward_batch + GAMMA * not_done_mask  * target_Q).float()
        
        # Critic update
        current_Q1, current_Q2 = self.critic(state_batch, action_batch)
        
        critic_loss = F.mse_loss(current_Q1, target_Q.unsqueeze(1)) + F.mse_loss(current_Q2, target_Q.unsqueeze(1))

        # Optimize the critic
        self.critic_optimizer.zero_grad()
        critic_loss.backward()
        self.critic_optimizer.step()

        # Delayed policy updates
        if self.total_it % self.policy_freq == 0:
            # Compute actor loss
            actor_loss = -self.critic.Q1(state_batch, self.actor(state_batch)).mean()
            
            # Optimize the actor 
            self.actor_optimizer.zero_grad()
            actor_loss.backward()
            self.actor_optimizer.step()

            # Target update
            soft_update(self.actor_target, self.actor, self.tau)
            soft_update(self.critic_target, self.critic, self.tau)

    def eval(self):
        self.actor.eval()
        self.actor_target.eval()
        self.critic.eval()
        self.critic_target.eval()

    def observe(self, r_t, s_t1, done):
        if self.is_training:
            self.memory.store(self.s_t, self.a_t, s_t1, r_t, done)
            self.s_t = s_t1

    def random_action(self):
        action = torch.tensor([np.random.uniform(-1.,1.,self.nb_actions)], device=device, dtype=torch.float)
        self.a_t = action
        
        return action

    def select_action(self, s_t, decay_epsilon=True):
        with torch.no_grad():
            action = self.actor(s_t).squeeze(0)
            action += torch.from_numpy(self.is_training * max(self.epsilon, 0) * self.random_process.sample()).to(device).float()
            action = torch.clamp(action, -1., 1.)

            action = action.unsqueeze(0)
            
            if decay_epsilon:
                self.epsilon -= self.depsilon
            
            self.a_t = action
            return action

    def reset(self, obs):
        self.s_t = obs
        self.random_process.reset_states()

In [10]:
def train_model():
    n_observations = env.observation_space.shape[0]
    n_actions = env.action_space.shape[0]
    
    agent = TD3(n_observations, n_actions).to(device)
    
    max_episode_length = 500
    
    agent.is_training = True
    episode_reward = 0.
    observation = None
    
    warmup = 50
    num_episodes = 2000 # M
    episode_durations = []

    steps = 0

    for i_episode in range(num_episodes):
        observation = env.reset()
        state = torch.from_numpy(observation).float().unsqueeze(0).to(device)
        agent.reset(state)
        
        overall_reward = 0
        episode_steps = 0
        done = False
        while not done:
            # agent pick action ...
            if i_episode <= warmup:
                action = agent.random_action()
            else:
                action = agent.select_action(state)

            # env response with next_observation, reward, terminate_info
            observation, reward, done, info = env.step(action.detach().cpu().squeeze(0).numpy())
            steps += 1
            
            if max_episode_length and episode_steps >= max_episode_length -1:
                done = True
                
            extrinsic_reward = torch.tensor([reward], device=device)
            
            overall_reward += reward
            
            next_state = torch.from_numpy(observation).float().unsqueeze(0).to(device)

            # agent observe and update policy
            agent.observe(extrinsic_reward, next_state, done)

            episode_steps += 1            

            state = next_state
            
            if i_episode > warmup:
                agent.update_policy()

        episode_durations.append((i_episode, overall_reward))
        #plot_durations(episode_durations)
        _, dur = list(map(list, zip(*episode_durations)))
        if len(dur) > 100:
            if np.mean(dur[-100:]) >= 195:
                print(f"Solved after {i_episode} episodes!")
                return agent
    return agent

In [14]:
state_max = torch.from_numpy(env.observation_space.high).to(device)
def eval_model(agent, episode_durations):
    agent.eval()

    max_episode_length = 200
    num_episodes = 100

    for noise in np.arange(0,0.31,0.03):

        overall_reward = 0
        for i_episode in range(num_episodes):
            observation = env.reset()
            # unsqueeze adds batch dimension
            state = torch.from_numpy(observation).float().unsqueeze(0).to(device)

            episode_steps = 0
            done = False
            while not done:
                state = state + state_max * torch.FloatTensor(state.shape).uniform_(-noise/2, noise/2).to(device)
                state = state.float()

                action = agent.select_action(state)
                observation, reward, done, _ = env.step(action.detach().cpu().squeeze(0).numpy())
                overall_reward += reward

                if max_episode_length and episode_steps >= max_episode_length - 1:
                    done = True
                episode_steps += 1

                state = torch.from_numpy(observation).float().unsqueeze(0).to(device)

        episode_durations[noise].append(overall_reward / num_episodes)

In [12]:
def plot_norms(episode_durations):
    plt.figure(2, figsize=(10,10))
    
    x, ys = np.array(list(episode_durations.keys())), np.array(list(episode_durations.values()))
    
    plt.title('Action Prediction $\mu$ and $\pm \sigma$ interval')
    plt.xlabel('L2 Norm')
    plt.ylabel('Average Reward')
    
    mu = np.mean(ys, axis=1)
    plt.plot(x / 10, mu)
    stds = np.std(ys, axis = 1)
    plt.fill_between(x / 10, mu + stds , mu - stds, alpha=0.2)
        
    plt.pause(0.001)  # pause a bit so that plots are updated
    display.clear_output(wait=True)

In [15]:
episodes = {}
for l2norm in np.arange(0,0.31,0.03):
    episodes[l2norm] = []

# train 20 models for 200 steps, then eval them
for i in range(20):
    agent = train_model()
    eval_model(agent, episodes)
    print(i, episodes)
    plot_norms(episodes)

plot_norms(episodes)
print(episodes)

{0.0: [199.96, 195.21, 200.0, 199.13, 200.0, 200.0, 200.0, 189.13, 197.08, 199.88, 194.55, 190.67, 199.92, 158.1, 200.0, 197.94, 199.48, 191.84, 200.0, 169.8], 0.03: [114.3, 36.53, 109.54, 181.7, 103.88, 32.98, 125.95, 49.81, 156.45, 39.77, 154.84, 189.48, 75.67, 88.33, 76.32, 61.9, 199.52, 47.24, 83.42, 182.34], 0.06: [44.66, 35.35, 46.66, 92.69, 40.73, 29.54, 60.17, 34.51, 64.16, 33.38, 54.87, 132.21, 37.21, 44.4, 37.46, 31.08, 188.41, 30.33, 43.91, 137.44], 0.09: [32.73, 29.24, 41.25, 51.72, 32.98, 23.44, 47.9, 25.82, 41.79, 30.17, 39.23, 79.74, 31.09, 34.84, 37.19, 29.9, 139.93, 29.84, 36.15, 81.43], 0.12: [31.22, 24.25, 30.37, 35.86, 28.04, 26.34, 43.63, 29.31, 34.03, 26.86, 35.54, 47.17, 27.37, 31.69, 31.06, 27.27, 90.83, 24.77, 31.58, 55.22], 0.15: [26.47, 26.01, 26.39, 30.36, 27.17, 23.71, 35.43, 23.95, 27.73, 26.45, 31.1, 42.11, 24.85, 27.0, 27.69, 24.1, 71.58, 24.07, 24.62, 47.36], 0.18: [26.0, 26.29, 26.05, 34.3, 27.74, 23.32, 34.1, 26.82, 29.81, 25.18, 28.53, 33.61, 24.07, 

In [16]:
import json
json = json.dumps(episodes)
f = open("td3_episodes.json","w")
f.write(json)
f.close()